In [1]:
from aoc import read_file

In [188]:
lines = read_file('20221207.txt').split('\n')

commands = [line.split(' ') for line in lines]

In [189]:
import json

def parse_commands(commands):
    i = 1

    root = {
        'name': '/',
        'path': '/',
        'type': 'dir',
        'contents': {}
    }

    dir_stack = [root]

    current_dir = root

    while i < len(lines):
        command = commands[i]
        if command[0] != '$':
            raise Exception('Should see command: {lines[index]}')
        if command[1] == 'cd':
            if command[2] == '..':
                current_dir = dir_stack.pop()
            else:
                directory_name = command[2]
                dir_stack.append(current_dir)
                current_dir = current_dir['contents'][directory_name]
        elif command[1] == 'ls':
            while i+1 < len(commands) and commands[i+1][0] != '$':
                i += 1
                if commands[i][0] == 'dir':
                    directory_name = commands[i][1]
                    current_dir['contents'][directory_name] = {
                        'name': directory_name,
                        'path': current_dir['path'] + '/' + directory_name,
                        'type': 'dir',
                        'contents': {}
                    }
                else:
                    size = int(commands[i][0])
                    file_name = commands[i][1]
                    current_dir['contents'][file_name] = {
                        'name': file_name,
                        'type': 'file',
                        'size': size
                    }
        i += 1

    return root

In [203]:
directory_tree = parse_commands(commands)

In [197]:
def get_directory_sizes(directory):
    sizes = {
        'name': directory['name'],
        'path': directory['path'],
        'contents': {}
    }
    
    dir_size = 0
    
    for name, content_file in directory['contents'].items():
        if content_file['type'] == 'file':
            dir_size += content_file['size']
        else:
            subdir_size = get_directory_sizes(content_file)
            sizes['contents'][name] = subdir_size
            dir_size += subdir_size['size']

    sizes['size'] = dir_size
    return sizes

In [199]:
directory_sizes = get_directory_sizes(directory_tree)

In [193]:
def get_directory_size_map(directory_sizes):
    size_map = {}
    
    def get_directory_size_map_rec(directory):
        size_map[directory['path']] = directory['size']
        for sub_dir in directory['contents'].values():
            get_directory_size_map_rec(sub_dir)
    
    get_directory_size_map_rec(directory_sizes)
    
    return size_map

In [200]:
size_map = get_directory_size_map(directory_sizes)

In [201]:
sum([size for path, size in size_map.items() if size <= 100000])

1491614

In [202]:
available_space = 40000000

used_space = size_map['/']

required_space = used_space - available_space

for size in sorted([size for path, size in size_map.items()]):
    if size < required_space:
        continue
    else:
        print(size)
        break

6400111
